In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import random as rn
from keras.layers import Dense
from keras import Input
#from keras.engine import Model
from keras.layers import Dense, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, Multiply, Conv2D, MaxPooling2D
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from sklearn.preprocessing import MinMaxScaler

In [2]:
data_dir = 'C:/Users/USER/Desktop/spcup_2022/audiofile/'

In [21]:
X_real = []
X_imag = []
Y = []
X_shape_list = []
categories = ["0","1","2","3","4"]
num_classes = len(categories)


for filename in os.listdir(f'C:/Users/USER/Desktop/spcup_2022/audiofile/'):
    audioname = f'C:/Users/USER/Desktop/spcup_2022/audiofile/{filename}'
    y, sr = librosa.load(audioname, duration=1, sr = 16000)  # 모두 1초까지만 자름
    stft = librosa.stft(y, n_fft=512, hop_length=384)   # 일반적으로 1/4 가량 겹치게 함
    real = stft.real
    imag = stft.imag
    
    scaler = StandardScaler()
    scaler.fit(real)
    real = scaler.transform(real)
    
    scaler = StandardScaler()
    scaler.fit(imag)
    imag = scaler.transform(imag)
    
    X_real.append(real)
    X_imag.append(imag)   
    
for idex, category in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    for j in range(1000):
        Y.append(label)

X_real = np.array(X_real)
X_imag = np.array(X_imag)
Y = np.array(Y)
print(X_real.shape)
print(X_imag.shape)
print(Y.shape)

(5000, 257, 42)
(5000, 257, 42)
(5000, 5)


In [ ]:
X[0].shape, X[1].shape, X[318].shape

In [22]:
X_real_train = np.concatenate((X_real[0:800],X_real[1000:1800],X_real[2000:2800],X_real[3000:3800],X_real[4000:4800]), axis = 0)
X_real_test = np.concatenate((X_real[800:1000],X_real[1800:2000],X_real[2800:3000],X_real[3800:4000],X_real[4800:5000]), axis = 0)
Y_real_train = np.concatenate((Y[0:800],Y[1000:1800],Y[2000:2800],Y[3000:3800],Y[4000:4800]), axis = 0)
Y_real_test = np.concatenate((Y[800:1000],Y[1800:2000],Y[2800:3000],Y[3800:4000],Y[4800:5000]), axis = 0)

X_imag_train = np.concatenate((X_imag[0:800],X_imag[1000:1800],X_imag[2000:2800],X_imag[3000:3800],X_imag[4000:4800]), axis = 0)
X_imag_test = np.concatenate((X_imag[800:1000],X_imag[1800:2000],X_imag[2800:3000],X_imag[3800:4000],X_imag[4800:5000]), axis = 0)
Y_imag_train = np.concatenate((Y[0:800],Y[1000:1800],Y[2000:2800],Y[3000:3800],Y[4000:4800]), axis = 0)
Y_imag_test = np.concatenate((Y[800:1000],Y[1800:2000],Y[2800:3000],Y[3800:4000],Y[4800:5000]), axis = 0)

print('X_real')
print(X_real_train.shape)
print(X_real_test.shape)
print(Y_real_train.shape)
print(Y_real_test.shape)
print('X_imag')
print(X_imag_train.shape)
print(X_imag_test.shape)
print(Y_imag_train.shape)
print(Y_imag_test.shape)

X_real
(4000, 257, 42)
(1000, 257, 42)
(4000, 5)
(1000, 5)
X_imag
(4000, 257, 42)
(1000, 257, 42)
(4000, 5)
(1000, 5)


In [23]:
# CNN 모델에 들어가려면 4차원으로 바꿔줘야함
X_real_train = X_real_train.reshape(-1,257,42,1)
X_real_test = X_real_test.reshape(-1,257,42,1)

X_imag_train = X_imag_train.reshape(-1,257,42,1)
X_imag_test = X_imag_test.reshape(-1,257,42,1)

In [24]:
from keras.models import Sequential
from keras.layers import Flatten, Conv2D, MaxPooling2D

In [29]:
model = Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(257, 42, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dense(32, activation='relu'))
model.add(Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [30]:
# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
classifier = model.fit(X_imag_train,
                    Y_imag_train,
                    epochs=10,
                    batch_size=64)

Epoch 1/10
63/63 [==============================] - 30s 472ms/step - loss: 1.1103 - accuracy: 0.5807
Epoch 2/10
63/63 [==============================] - 29s 457ms/step - loss: 0.5748 - accuracy: 0.8518
Epoch 3/10
63/63 [==============================] - 30s 478ms/step - loss: 0.4178 - accuracy: 0.9333
Epoch 4/10
63/63 [==============================] - 30s 478ms/step - loss: 0.3494 - accuracy: 0.9620
Epoch 5/10
63/63 [==============================] - 29s 467ms/step - loss: 0.2980 - accuracy: 0.9803
Epoch 6/10
63/63 [==============================] - 29s 456ms/step - loss: 0.2656 - accuracy: 0.9902
Epoch 7/10
63/63 [==============================] - 31s 488ms/step - loss: 0.2441 - accuracy: 0.9930
Epoch 8/10
63/63 [==============================] - 29s 463ms/step - loss: 0.2261 - accuracy: 0.9960
Epoch 9/10
63/63 [==============================] - 30s 470ms/step - loss: 0.2122 - accuracy: 0.9955
Epoch 10/10
63/63 [==============================] - 29s 462ms/step - loss: 0.2000 - accura

In [ ]:
plt.plot(classifier.history['loss'])
#plt.plot(classifier.history['accuracy'])
plt.legend(['training', 'validation'], loc = 'upper left')
plt.show()

In [28]:
test_loss, test_acc = model.evaluate(X_real_test,  Y_real_test, verbose=2)
print('테스트 정확도:', test_acc)

32/32 - 2s - loss: 0.2907 - accuracy: 0.9130 - 2s/epoch - 71ms/step
테스트 정확도: 0.9129999876022339


In [32]:
test_loss, test_acc = model.evaluate(X_imag_test,  Y_imag_test, verbose=2)
print('테스트 정확도:', test_acc)

32/32 - 2s - loss: 0.4945 - accuracy: 0.8880 - 2s/epoch - 72ms/step
테스트 정확도: 0.8880000114440918


In [33]:
prediction = model.predict(X_imag_test)

In [34]:
predicted_classes = np.argmax(prediction, axis = 1)
print(predicted_classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 2
 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2
 1 1 2 2 2 2 2 2 2 1 2 2 2 2 1 1 2 2 2 2 2 1 1 1 1 1 2 2 1 2 1 2 2 1 2 2 2
 1 2 1 1 2 1 1 1 2 2 2 2 2 1 1 1 2 2 1 2 2 1 2 2 2 1 2 2 2 2 1 3 1 1 2 2 2
 2 2 2 2 2 1 2 2 1 2 2 1 

In [36]:
labels = np.argmax(Y_imag_test, axis=1)
print(labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [ ]:
labels.shape, predicted_classes.shape

In [ ]:
print(np.where(predicted_classes[0] != labels[0]))

In [ ]:
index = np.where(predicted_classes != labels)
print(index)

In [ ]:
len(index[0])

In [ ]:
index[0][0]

In [ ]:
X_filename = []
for filename in os.listdir(f'C:/Users/USER/Desktop/spcup_2022/audiofile/'):
    audioname = f'C:/Users/USER/Desktop/spcup_2022/audiofile/{filename}'
    X_filename.append(audioname)
X_filename = np.array(X_filename)

In [ ]:
len(X_filename)

In [ ]:
X_test_filename = np.concatenate((X_filename[800:1000],X_filename[1800:2000],X_filename[2800:3000],X_filename[3800:4000],X_filename[4800:5000]), axis = 0)

In [ ]:
len(X_test_filename)

In [ ]:
X_test_filename[0]

In [ ]:
# 잘못 예측한 오디오 파일들 
wrong_predictions = []

for i in range(len(index[0])):
    wrong_predictions.append(X_test_filename[index[0][i]])
    print(X_test_filename[index[0][i]])
    
print(len(wrong_predictions))

In [ ]:
# 잘못 예측한거 plot
for audio in wrong_predictions:
    y, sr = librosa.load(audio, duration=1)  # 모두 1초까지만 자름
    stft = np.abs(librosa.stft(y, n_fft=512, hop_length=384))   # 일반적으로 1/4 가량 겹치게 함
    spectogram = np.abs(stft)
    plt.figure(figsize=(10,5))
    plt.title(audio)
    plt.plot(spectogram) 

In [ ]:
# 잘못 예측한 오디오 파일들 모은 폴더 생성
import shutil
import os
 
file_source = 'C:/Users/USER/Desktop/spcup_2022/audiofile/'
file_destination = 'C:/Users/USER/Desktop/wrong_prediction'
 
get_files = os.listdir(file_source)
 

for i in range(len(wrong_predictions)):
    print(wrong_predictions[i][43:])
    for g in get_files:
        if g == wrong_predictions[i][43:]:
            shutil.move(file_source + g, file_destination)